In [1]:
#full printing
def print_full(x):
    pd.set_option('display.max_rows', len(x))
    print(x)
    pd.reset_option('display.max_rows')

def print_how_many(x, rows):
    pd.set_option('display.max_rows', rows)
    print(x)
    pd.reset_option('display.max_rows')

In [2]:
#data editing in dataframes
def edit_pop(x):
    x = x.replace("\t", "")
    x = x.replace(" ", "")
    x = x.replace("Woj.", "")
    x = x.replace("M.", "")
    return x

def edit_inf(x):
    x = x.replace("Powiat ", "")
    x = x.replace(" ", "")
    x = x.replace("st.", "")
    x = x.replace("m.", "")
    return x

def edit_new(x):
    x = x.replace("\t", "")
    x = x.replace(" ", "")
    x = x.replace("st.", "")
    x = x.replace("m.", "")
    x = x.replace("Woj.", "")
    return x

In [3]:
#dopasowanie liniowe y=ax+b, scipy
def fit_linlin_scipy(x, y):

    idx = np.isnan(y)
    slope, intercept, r_value, p_value, std_err = stats.linregress(x[~idx],y[~idx])
    
    return slope, intercept, r_value, p_value

In [4]:
#dopasowanie liniowe y=ax, statsmodels, aby zmienić na y=ax+b trzebaby odkomentować linijkę i zmienić trochę returna
def fit_linlin_statsmodels(x, y):

    idx = np.isnan(y)
    #x = sm.add_constant(x) #comment to not include intercept
    model = sm.OLS(y[~idx],x[~idx])
    results = model.fit()
    S = (results.ssr/results.df_resid)**(1/2)
    S_div = S/y[~idx].mean()
    S_med = S/y[~idx].median()
    

    return results.params[0], 0, results.bse[0], results.rsquared, results.pvalues[0], S, S_div, S_med

In [5]:
#dopasowanie liniowe do wyników w skali logarytmicznej log(y)=a*log(x)+b, scipy
def fit_loglog_scipy(x, y):

    idx = np.isnan(y)
    xlog = np.log10(x)
    ylog = np.log10(y)

    slope, intercept, r_value, p_value, std_err = stats.linregress(xlog[~idx],ylog[~idx])
    
    return slope, intercept, r_value, p_value

In [6]:
def fit_loglog_statsmodels(x, y):

    idx = np.isnan(y)
    y = y[~idx]
    x = x[~idx]
    idx = np.isnan(x)
    y = y[~idx]
    x = x[~idx]
    
    xlog = np.log10(x)
    ylog = np.log10(y)
    x = sm.add_constant(x)
    xlog = sm.add_constant(xlog)
    
    
    model = sm.OLS(ylog,xlog)
    results = model.fit()
    y_pred = pow(10, results.predict(xlog))
    ssr = np.sum(pow(y-y_pred, 2))
    S = (ssr/results.df_resid)**(1/2)
    S_div = S/y.mean()
    S_med = S/y.median()
    
    
    return results.params[1], results.params[0], results.bse[1], results.rsquared, results.pvalues[1], S, S_div, S_med

In [7]:
def fit_pow(x, y):

    idx = np.isnan(y)
    y = y[~idx]
    x = x[~idx]
    idx = np.isnan(x)
    y = y[~idx]
    x = x[~idx]
    
    popt, pcov = curve_fit(power_func, x, y)
    ssr = np.sum(pow(y-power_func(x, *popt), 2))
    S_pow = (ssr/(x.size-2))**(1/2)
    S_pow_div = S_pow/y.mean()
    S_pow_med = S_pow/y.median()    
    
    return popt[1], popt[0], S_pow_med

In [8]:
#dopasowanie liniowe y=ax+b i wykres, scipy
# def fit_n_plot_linlin_scipy(x, y, title, xscale_tabel, yscale_label):

#     plt.plot(x, y, 'bo', markersize=3)
#     plt.title(title)
#     plt.xlabel(xscale_tabel)
#     plt.ylabel(yscale_label)

#     idx = np.isnan(y)
    
#     slope, intercept, r_value, p_value, std_err = stats.linregress(x[~idx],y[~idx])
#     plt.plot(x, slope * x + intercept, color='red')
#     return slope, intercept, r_value, p_value, std_err

In [9]:
#dopasowanie liniowe y=ax i wykres, statsmodels
def fit_n_plot_linlin_statsmodels(x, y, title, xscale_tabel, yscale_label):
    
    idx = np.isnan(y)
    x = x[~idx]
    y = y[~idx]
    plt.plot(x, y, 'bo', markersize=3)
    plt.title(title)
    plt.xlabel(xscale_tabel)
    plt.ylabel(yscale_label)
    
    #x = sm.add_constant(x[~idx]) #comment to not include intercept
    model = sm.OLS(y,x)
    results = model.fit()
    S = (results.ssr/results.df_resid)**(1/2)
    S_div = S/y.mean()
    S_med = S/y.median()
    
    plt.plot(x, results.predict(x), color='red')
    
    return results.params[0], 0, results.bse[0], results.rsquared, results.pvalues[0], S, S_div, S_med

In [10]:
#dopasowanie liniowe y=ax+b i wykres, statsmodels
def fit_n_plot_linlin_statsmodels_intercept(x, y, title, xscale_tabel, yscale_label):
    
    idx = np.isnan(y)
    plt.plot(x[~idx], y[~idx], 'bo', markersize=3)
    plt.title(title)
    plt.xlabel(xscale_tabel)
    plt.ylabel(yscale_label)
    
    x = sm.add_constant(x[~idx]) #comment to not include intercept
    y = y[~idx]
    model = sm.OLS(y,x)
    results = model.fit()
    S = (results.ssr/results.df_resid)**(1/2)
    S_div = S/y.mean()
    S_med = S/y.median()
    
    plt.plot(x.loc[:, x.columns != 'const'], results.predict(x), color='red')
    
    return results.params[1], results.params[0], results.bse[1], results.rsquared, results.pvalues[1], S, S_div, S_med

In [11]:
#dopasowanie liniowe do wyników w skali logarytmicznej log(y)=a*log(x)+b i wykres, scipy
def fit_n_plot_loglog_scipy(x, y, title, xscale_label, yscale_label):

    plt.plot(x, y, 'bo', markersize=3)
    plt.title(title)
    plt.yscale("log")
    plt.xscale("log")
    plt.xlabel(xscale_label)
    plt.ylabel(yscale_label)

    idx = np.isnan(y)
    xlog = np.log10(x)
    ylog = np.log10(y)
    
    slope, intercept, r_value, p_value, std_err = stats.linregress(xlog[~idx],ylog[~idx])
    plt.plot(x, pow(x, slope) * pow(10, intercept), color='red')
    return slope, intercept, r_value, p_value, std_err

In [12]:
#dopasowanie liniowe do wyników w skali logarytmicznej log(y)=a*log(x)+b i wykres, statsmodels
def fit_n_plot_loglog_statsmodels(x, y, title, xscale_label, yscale_label):
    
    idx = np.isnan(y)
    y = y[~idx]
    x = x[~idx]
    #plt.plot(x, y, 'ro', markersize=3)
    plt.plot(x, y, 'ro')
    plt.title(title)
    plt.yscale("log")
    plt.xscale("log")
    plt.xlabel(xscale_label)
    plt.ylabel(yscale_label)
    
    xlog = np.log10(x)
    ylog = np.log10(y)
    x = sm.add_constant(x)
    xlog = sm.add_constant(xlog)
    
    model = sm.OLS(ylog,xlog)
    results = model.fit()
    y_pred = pow(10, results.predict(xlog))
    ssr = np.sum(pow(y-y_pred, 2))
    S = (ssr/results.df_resid)**(1/2)
    S_div = S/y.mean()
    S_med = S/y.median()
    plt.plot(x.loc[:, x.columns != 'const'], pow(10, results.predict(xlog)), color='blue', label='$\lambda_i$ = '+ "{:.2f}".format(results.params[1])+'$\pm$'+"{:.2f}".format(results.bse[1])+', $S_{med}$ = '+"{:.2f}".format(S_med))
    plt.legend()
    
    return results.params[1], results.params[0], results.bse[1], results.rsquared, results.pvalues[1], S, S_div, S_med

In [13]:
#dopasowanie liniowe do wyników w skali logarytmicznej log(y)=a*log(x)+b i wykres, statsmodels
# to samo co wyżej ale 4 ploty na jednym canvas
def fit_n_plot_loglog_statsmodels_x4(x1, y1, x2, y2, x3, y3, x4, y4, title1, title2, title3, title4, xscale_label, yscale_label):
    
    fig, axs = plt.subplots(2, 2)
    
    idx = np.isnan(y1)
    y1 = y1[~idx]
    x1 = x1[~idx]
    idx2 = np.isnan(x1)
    y1 = y1[~idx2]
    x1 = x1[~idx2]
    
    axs[0, 0].plot(x1, y1, 'o', color = 'black', markersize=3)
    axs[0, 0].set_title(title1)
    axs[0, 0].set_yscale("log")
    axs[0, 0].set_xscale("log")
    #axs[0, 0].set_ylim([2/1000, 1.01/10])
    #axs[0, 0].set_xlim([2, 200])
    
    xlog1 = np.log10(x1)
    ylog1 = np.log10(y1)
    x1 = sm.add_constant(x1)
    xlog1 = sm.add_constant(xlog1)
    
    model = sm.OLS(ylog1,xlog1)
    results = model.fit()
    y_pred = pow(10, results.predict(xlog1))
    ssr = np.sum(pow(y1-y_pred, 2))
    S = (ssr/results.df_resid)**(1/2)
    S_div1 = S/y1.mean()
    S_med1 = S/y1.median()
    
    axs[0, 0].plot(x1.loc[:, x1.columns != 'const'], pow(10, results.predict(xlog1)), color='blue', label='$\lambda_d$ = '+ "{:.2f}".format(results.params[1])+'$\pm$'+"{:.2f}".format(results.bse[1])+', $S_{med}$ = '+"{:.2f}".format(S_med1))
    axs[0, 0].legend(loc='lower left')
    

    
    idx = np.isnan(y2)
    y2 = y2[~idx]
    x2 = x2[~idx]
    idx2 = np.isnan(x2)
    y2 = y2[~idx2]
    x2 = x2[~idx2]
    
    axs[0, 1].plot(x2, y2, 'o', color = 'black', markersize=3)
    axs[0, 1].set_title(title2)
    axs[0, 1].set_yscale("log")
    axs[0, 1].set_xscale("log")
    axs[0, 1].set_ylim([1/10000, 3/1000])
    #axs[0, 1].set_ylim(axs[0, 0].get_ylim())
    
    xlog2 = np.log10(x2)
    ylog2 = np.log10(y2)
    x2 = sm.add_constant(x2)
    xlog2 = sm.add_constant(xlog2)
    
    model = sm.OLS(ylog2,xlog2)
    results = model.fit()
    y_pred = pow(10, results.predict(xlog2))
    ssr = np.sum(pow(y2-y_pred, 2))
    S = (ssr/results.df_resid)**(1/2)
    S_div2 = S/y2.mean()
    S_med2 = S/y2.median()
    
    axs[0, 1].plot(x2.loc[:, x2.columns != 'const'], pow(10, results.predict(xlog2)), color='blue', label='$\lambda_d$ = '+ "{:.2f}".format(results.params[1])+'$\pm$'+"{:.2f}".format(results.bse[1])+', $S_{med}$ = '+"{:.2f}".format(S_med2))
    axs[0, 1].legend(loc='lower left')
    
    
    
    idx = np.isnan(y3)
    y3 = y3[~idx]
    x3 = x3[~idx]
    idx2 = np.isnan(x3)
    y3 = y3[~idx2]
    x3 = x3[~idx2]
    
    axs[1, 0].plot(x3, y3, 'o', color = 'black', markersize=3)
    axs[1, 0].set_title(title3)
    axs[1, 0].set_yscale("log")
    axs[1, 0].set_xscale("log")
    #axs[1, 0].set_ylim([9/1000, 1.2/10])
    axs[1, 0].set_ylim(axs[0, 1].get_ylim())
    #axs[1, 0].set_xlim(axs[0, 0].get_xlim())
    
    xlog3 = np.log10(x3)
    ylog3 = np.log10(y3)
    x3 = sm.add_constant(x3)
    xlog3 = sm.add_constant(xlog3)
    
    model = sm.OLS(ylog3,xlog3)
    results = model.fit()
    y_pred = pow(10, results.predict(xlog3))
    ssr = np.sum(pow(y3-y_pred, 2))
    S = (ssr/results.df_resid)**(1/2)
    S_div3 = S/y3.mean()
    S_med3 = S/y3.median()
    
    axs[1, 0].plot(x3.loc[:, x3.columns != 'const'], pow(10, results.predict(xlog3)), color='blue', label='$\lambda_d$ = '+ "{:.2f}".format(results.params[1])+'$\pm$'+"{:.2f}".format(results.bse[1])+', $S_{med}$ = '+"{:.2f}".format(S_med3))
    axs[1, 0].legend(loc='lower left')
    
    
    
    idx = np.isnan(y4)
    y4 = y4[~idx]
    x4 = x4[~idx]
    idx2 = np.isnan(x4)
    y4 = y4[~idx2]
    x4 = x4[~idx2]
    
    axs[1, 1].plot(x4, y4, 'o', color = 'black', markersize=3)
    axs[1, 1].set_title(title4)
    axs[1, 1].set_yscale("log")
    axs[1, 1].set_xscale("log")
    #axs[1, 1].set_ylim([9/1000, 1.2/10])
    axs[1, 1].set_ylim(axs[0, 1].get_ylim())
    #axs[1, 1].set_ylim(axs[1, 0].get_ylim())
    
    
    xlog4 = np.log10(x4)
    ylog4 = np.log10(y4)
    x4 = sm.add_constant(x4)
    xlog4 = sm.add_constant(xlog4)
    
    model = sm.OLS(ylog4,xlog4)
    results = model.fit()
    y_pred = pow(10, results.predict(xlog4))
    ssr = np.sum(pow(y4-y_pred, 2))
    S = (ssr/results.df_resid)**(1/2)
    S_div4 = S/y4.mean()
    S_med4 = S/y4.median()
    
    axs[1, 1].plot(x4.loc[:, x4.columns != 'const'], pow(10, results.predict(xlog4)), color='blue', label='$\lambda_d$ = '+ "{:.2f}".format(results.params[1])+'$\pm$'+"{:.2f}".format(results.bse[1])+', $S_{med}$ = '+"{:.2f}".format(S_med4))
    axs[1, 1].legend(loc='lower left')
    #axs[1, 1].yaxis.set_major_formatter(matplotlib.ticker.FormatStrFormatter('%.2f'))
    
    for ax in axs.flat:
        ax.set(xlabel=xscale_label, ylabel=yscale_label)
        
#     for ax in axs.flat:
#         ax.label_outer()
        
    #plt.setp(axs[0, 0].get_xticklabels(), visible=False)
    #plt.setp(axs[0, 1].get_xticklabels(), visible=False)
    axs[0, 0].set(xlabel=None)
    axs[0, 1].set(xlabel=None)
    axs[0, 1].set(ylabel=None)
    axs[1, 1].set(ylabel=None)
    
    plt.tight_layout()
    
    return results.params[1], results.params[0], results.bse[1], results.rsquared, results.pvalues[1], S, S_div, S_med

In [14]:
#dopasowanie liniowe do wyników w skali logarytmicznej log(y)=a*log(x)+b i wykres, statsmodels
# to samo co wyżej ale 4 ploty na jednym canvas
def fit_n_plot_loglog_n_pow_x4(x1, y1, x2, y2, x3, y3, x4, y4, title1, title2, title3, title4, xscale_label, yscale_label):
    
    fig, axs = plt.subplots(2, 2)
    
    idx = np.isnan(y1)
    y1 = y1[~idx]
    x1 = x1[~idx]
    idx2 = np.isnan(x1)
    y1 = y1[~idx2]
    x1 = x1[~idx2]
    
    axs[0, 0].plot(x1, y1, 'o', color = 'black', markersize=3)
    axs[0, 0].set_title(title1)
    axs[0, 0].set_yscale("log")
    axs[0, 0].set_xscale("log")
    #axs[0, 0].set_ylim([2/1000, 1/10])
    #axs[0, 0].set_xlim([2, 200])
    
    popt, pcov = curve_fit(power_func, x1, y1)
    ssr2 = np.sum(pow(y1-power_func(x1, *popt), 2))
    S_pow1 = (ssr2/(x1.size-2))**(1/2)
    S_pow_div1 = S_pow1/y1.mean()
    S_pow_med1 = S_pow1/y1.median()    
    perr = np.sqrt(np.diag(pcov))
    axs[0, 0].plot(x1, power_func(x1, *popt), color='green', label='$\lambda_i$ = '+ "{:.2f}".format(popt[1])+'$\pm$'+"{:.2f}".format(perr[1])+', $S_{med}$ = '+"{:.2f}".format(S_pow_med1))
    
    xlog1 = np.log10(x1)
    ylog1 = np.log10(y1)
    x1 = sm.add_constant(x1)
    xlog1 = sm.add_constant(xlog1)
    
    model = sm.OLS(ylog1,xlog1)
    results = model.fit()
    y_pred = pow(10, results.predict(xlog1))
    ssr = np.sum(pow(y1-y_pred, 2))
    S = (ssr/results.df_resid)**(1/2)
    S_div1 = S/y1.mean()
    S_med1 = S/y1.median()
    
    axs[0, 0].plot(x1.loc[:, x1.columns != 'const'], pow(10, results.predict(xlog1)), color='blue', label='$\lambda_i$ = '+ "{:.2f}".format(results.params[1])+'$\pm$'+"{:.2f}".format(results.bse[1])+', $S_{med}$ = '+"{:.2f}".format(S_med1))
    axs[0, 0].legend(loc='upper left')
    

    
    idx = np.isnan(y2)
    y2 = y2[~idx]
    x2 = x2[~idx]
    idx2 = np.isnan(x2)
    y2 = y2[~idx2]
    x2 = x2[~idx2]
    
    axs[0, 1].plot(x2, y2, 'o', color = 'black', markersize=3)
    axs[0, 1].set_title(title2)
    axs[0, 1].set_yscale("log")
    axs[0, 1].set_xscale("log")
    #axs[0, 1].set_ylim([8/1000, 1/10])
    #axs[0, 1].set_ylim(axs[0, 0].get_ylim())
    
    popt, pcov = curve_fit(power_func, x2, y2)
    ssr2 = np.sum(pow(y2-power_func(x2, *popt), 2))
    S_pow2 = (ssr2/(x2.size-2))**(1/2)
    S_pow_div2 = S_pow2/y2.mean()
    S_pow_med2 = S_pow2/y2.median()
    perr = np.sqrt(np.diag(pcov))
    axs[0, 1].plot(x4, power_func(x4, *popt), color='green', label='$\lambda_i$ = '+ "{:.2f}".format(popt[1])+'$\pm$'+"{:.2f}".format(perr[1])+', $S_{med}$ = '+"{:.2f}".format(S_pow_med2))
    
    xlog2 = np.log10(x2)
    ylog2 = np.log10(y2)
    x2 = sm.add_constant(x2)
    xlog2 = sm.add_constant(xlog2)
    
    model = sm.OLS(ylog2,xlog2)
    results = model.fit()
    y_pred = pow(10, results.predict(xlog2))
    ssr = np.sum(pow(y2-y_pred, 2))
    S = (ssr/results.df_resid)**(1/2)
    S_div2 = S/y2.mean()
    S_med2 = S/y2.median()
    
    axs[0, 1].plot(x2.loc[:, x2.columns != 'const'], pow(10, results.predict(xlog2)), color='blue', label='$\lambda_i$ = '+ "{:.2f}".format(results.params[1])+'$\pm$'+"{:.2f}".format(results.bse[1])+', $S_{med}$ = '+"{:.2f}".format(S_med2))
    axs[0, 1].legend(loc='upper left')
    
    
    
    idx = np.isnan(y3)
    y3 = y3[~idx]
    x3 = x3[~idx]
    idx2 = np.isnan(x3)
    y3 = y3[~idx2]
    x3 = x3[~idx2]
    
    axs[1, 0].plot(x3, y3, 'o', color = 'black', markersize=3)
    axs[1, 0].set_title(title3)
    axs[1, 0].set_yscale("log")
    axs[1, 0].set_xscale("log")
    #axs[1, 0].set_ylim([8/1000, 1/10])
    #axs[1, 0].set_xlim(axs[0, 0].get_xlim())
    
    popt, pcov = curve_fit(power_func, x3, y3)
    ssr2 = np.sum(pow(y3-power_func(x3, *popt), 2))
    S_pow3 = (ssr2/(x3.size-2))**(1/2)
    S_pow_div3 = S_pow3/y3.mean()
    S_pow_med3 = S_pow3/y3.median()
    perr = np.sqrt(np.diag(pcov))
    axs[1, 0].plot(x4, power_func(x4, *popt), color='green', label='$\lambda_i$ = '+ "{:.2f}".format(popt[1])+'$\pm$'+"{:.2f}".format(perr[1])+', $S_{med}$ = '+"{:.2f}".format(S_pow_med3))
    
    xlog3 = np.log10(x3)
    ylog3 = np.log10(y3)
    x3 = sm.add_constant(x3)
    xlog3 = sm.add_constant(xlog3)
    
    model = sm.OLS(ylog3,xlog3)
    results = model.fit()
    y_pred = pow(10, results.predict(xlog3))
    ssr = np.sum(pow(y3-y_pred, 2))
    S = (ssr/results.df_resid)**(1/2)
    S_div3 = S/y3.mean()
    S_med3 = S/y3.median()
    
    axs[1, 0].plot(x3.loc[:, x3.columns != 'const'], pow(10, results.predict(xlog3)), color='blue', label='$\lambda_i$ = '+ "{:.2f}".format(results.params[1])+'$\pm$'+"{:.2f}".format(results.bse[1])+', $S_{med}$ = '+"{:.2f}".format(S_med3))
    axs[1, 0].legend(loc='upper left')
    
    
    
    idx = np.isnan(y4)
    y4 = y4[~idx]
    x4 = x4[~idx]
    idx2 = np.isnan(x4)
    y4 = y4[~idx2]
    x4 = x4[~idx2]
    
    axs[1, 1].plot(x4, y4, 'o', color = 'black', markersize=3)
    axs[1, 1].set_title(title4)
    axs[1, 1].set_yscale("log")
    axs[1, 1].set_xscale("log")
    #axs[1, 1].set_ylim([8/1000, 1.5/10])
    #axs[1, 1].set_ylim(axs[1, 0].get_ylim())
    
    popt, pcov = curve_fit(power_func, x4, y4)
    ssr2 = np.sum(pow(y4-power_func(x4, *popt), 2))
    S_pow4 = (ssr2/(x4.size-2))**(1/2)
    S_pow_div4 = S_pow4/y4.mean()
    S_pow_med4 = S_pow4/y4.median()
    perr = np.sqrt(np.diag(pcov))
    axs[1, 1].plot(x4, power_func(x4, *popt), color='green', label='$\lambda_i$ = '+ "{:.2f}".format(popt[1])+'$\pm$'+"{:.2f}".format(perr[1])+', $S_{med}$ = '+"{:.2f}".format(S_pow_med4))
    xlog4 = np.log10(x4)
    ylog4 = np.log10(y4)
    x4 = sm.add_constant(x4)
    xlog4 = sm.add_constant(xlog4)
    
    model = sm.OLS(ylog4,xlog4)
    results = model.fit()
    y_pred = pow(10, results.predict(xlog4))
    ssr = np.sum(pow(y4-y_pred, 2))
    S = (ssr/results.df_resid)**(1/2)
    S_div4 = S/y4.mean()
    S_med4 = S/y4.median()
    
    axs[1, 1].plot(x4.loc[:, x4.columns != 'const'], pow(10, results.predict(xlog4)), color='blue', label='$\lambda_i$ = '+ "{:.2f}".format(results.params[1])+'$\pm$'+"{:.2f}".format(results.bse[1])+', $S_{med}$ = '+"{:.2f}".format(S_med4))
    axs[1, 1].legend(loc='upper left')
    #axs[1, 1].yaxis.set_major_formatter(matplotlib.ticker.FormatStrFormatter('%.2f'))
    
    for ax in axs.flat:
        ax.set(xlabel=xscale_label, ylabel=yscale_label)
        
#     for ax in axs.flat:
#         ax.label_outer()
        
    #plt.setp(axs[0, 0].get_xticklabels(), visible=False)
    #plt.setp(axs[0, 1].get_xticklabels(), visible=False)
    axs[0, 0].set(xlabel=None)
    axs[0, 1].set(xlabel=None)
    axs[0, 1].set(ylabel=None)
    axs[1, 1].set(ylabel=None)
    
    plt.tight_layout()
    
    return results.params[1], results.params[0], results.bse[1], results.rsquared, results.pvalues[1], S, S_div, S_med

In [15]:
#dopasowanie liniowe do wyników w skali logarytmicznej log(y)=a*log(x)+b i liniowe y=ax na jednym wykresie, statsmodels
def fit_n_plot_lin_n_log_statsmodels(x, y, title, xscale_label, yscale_label):
    
    idx = np.isnan(y)
    y = y[~idx]
    x = x[~idx]
    
    plt.plot(x, y, 'bo', markersize=3, label='dane')
    plt.title(title)
    plt.yscale("log")
    plt.xscale("log")
    plt.xlabel(xscale_label)
    plt.ylabel(yscale_label)
    #plt.xlim([10000, plt.xlim()[1]])
    #plt.xlim([10000, 1000000])
    
    #x = sm.add_constant(x[~idx]) #comment to not include intercept
    model1 = sm.OLS(y,x)
    results1 = model1.fit()
    S_lin = (results1.ssr/results1.df_resid)**(1/2)
    print("ssr_lin = "+str(results1.ssr))
    print("df_resid_lin = "+str(results1.df_resid))
    S_lin_div = S_lin/y.mean()
    S_lin_med = S_lin/y.median()
    #print("Mediana_lin = "+str(y.median()))
#     plt.plot(x, results1.predict(x), color='red', label='liniowa, $R^2 = $'+"{:.2f}".format(results1.rsquared))
    plt.plot(x, results1.predict(x), color='red', label='liniowa, $S_{med}$ = '+"{:.2f}".format(S_lin_med))
    
    
    xlog = np.log10(x)
    ylog = np.log10(y)
    x = sm.add_constant(x)
    xlog = sm.add_constant(xlog)
    
    model2 = sm.OLS(ylog,xlog)
    results2 = model2.fit()
    y_pred2 = pow(10, results2.predict(xlog))
    ssr2 = np.sum(pow(y-y_pred2, 2))
    S_log = (ssr2/results2.df_resid)**(1/2)
    print("ssr_log = "+str(ssr2))
    print("df_resid_log = "+str(results2.df_resid))
    S_log_div = S_log/y.mean()
    S_log_med = S_log/y.median()
    #print("Mediana_log = "+str(y.median()))
    
    plt.plot(x.loc[:, x.columns != 'const'], y_pred2, color='black', label='log, $S_{med}$ = '+"{:.2f}".format(S_log_med)+', $\lambda$ = '+ "{:.2f}".format(results2.params[1])+'$\pm$'+"{:.2f}".format(results2.bse[1]))
    plt.legend()
    
    return results2.params[1], results2.params[0], results2.bse[1], results1.rsquared, results2.rsquared, S_lin, S_log, S_lin_div, S_log_div, S_lin_med, S_log_med

In [ ]:
#dopasowanie liniowe do wyników w skali logarytmicznej log(y)=a*log(x)+b i liniowe y=ax na jednym wykresie, statsmodels
def fit_n_plot_lin_n_log_statsmodels_new(x, y, title, xscale_label, yscale_label):
    
    idx = np.isnan(y)
    y = y[~idx]
    x = x[~idx]
    xlog = np.log10(x)
    ylog = np.log10(y)
    
    plt.plot(x, y, 'bo', markersize=3, label='dane')
    plt.title(title)
    plt.yscale("log")
    plt.xscale("log")
    plt.xlabel(xscale_label)
    plt.ylabel(yscale_label)
    #plt.xlim([10000, plt.xlim()[1]])
    #plt.xlim([10000, 1000000])
    
    #x = sm.add_constant(x[~idx]) #comment to not include intercept
    model1 = sm.OLS(y,x)
    results1 = model1.fit()
    y_pred_log = np.log10(results1.predict(x))
    ssr1 = np.sum(pow(ylog-y_pred_log, 2))
    S_lin = (ssr1/results1.df_resid)**(1/2)
    
    print("ssr_lin = "+str(results1.ssr))
    print("df_resid_lin = "+str(results1.df_resid))
    S_lin_div = S_lin/y.mean()
    S_lin_med = S_lin/y.median()
    #print("Mediana_lin = "+str(y.median()))
#     plt.plot(x, results1.predict(x), color='red', label='liniowa, $R^2 = $'+"{:.2f}".format(results1.rsquared))
    plt.plot(x, results1.predict(x), color='red', label='liniowa, $S_{med}$ = '+"{:.2f}".format(S_lin_med))
    
    

    x = sm.add_constant(x)
    xlog = sm.add_constant(xlog)
    
    model2 = sm.OLS(ylog,xlog)
    results2 = model2.fit()
    y_pred2 = pow(10, results2.predict(xlog))
    y_pred2_log = results2.predict(xlog)
    ssr2 = np.sum(pow(ylog-y_pred2_log, 2))
    S_log = (ssr2/results2.df_resid)**(1/2)
    print("ssr_log = "+str(ssr2))
    print("df_resid_log = "+str(results2.df_resid))
    S_log_div = S_log/y.mean()
    S_log_med = S_log/y.median()
    #print("Mediana_log = "+str(y.median()))
    
    plt.plot(x.loc[:, x.columns != 'const'], y_pred2, color='black', label='log, $S_{med}$ = '+"{:.2f}".format(S_log_med)+', $\lambda$ = '+ "{:.2f}".format(results2.params[1])+'$\pm$'+"{:.2f}".format(results2.bse[1]))
    plt.legend()
    
    return results2.params[1], results2.params[0], results2.bse[1], results1.rsquared, results2.rsquared, S_lin, S_log, S_lin_div, S_log_div, S_lin_med, S_log_med

In [16]:
def power_func(x, B, lmbda):
        return B*(x**lmbda)

In [17]:
#dopasowanie liniowe do wyników w skali logarytmicznej log(y)=a*log(x)+b i liniowe y=ax i liniowe y=B*x^lambda na jednym wykresie, statsmodels
def fit_n_plot_lin_n_log_n_pow(x, y, title, xscale_label, yscale_label):
    
    idx = np.isnan(y)
    y = y[~idx]
    x = x[~idx]
    
    plt.plot(x, y, 'bo', markersize=3, label='dane')
    plt.title(title)
    plt.yscale("log")
    plt.xscale("log")
    plt.xlabel(xscale_label)
    plt.ylabel(yscale_label)
    
    #x = sm.add_constant(x[~idx]) #comment to not include intercept
    model1 = sm.OLS(y,x)
    results1 = model1.fit()
    S_lin = (results1.ssr/results1.df_resid)**(1/2)
    print("ssr_lin = "+str(results1.ssr))
    print("df_resid_lin = "+str(results1.df_resid))
    S_lin_div = S_lin/y.mean()
    S_lin_med = S_lin/y.median()
    print("S_lin_med = "+str(S_lin_med))
    #print("Mediana_lin = "+str(y.median()))
#     plt.plot(x, results1.predict(x), color='red', label='liniowa, $R^2 = $'+"{:.2f}".format(results1.rsquared))
    plt.plot(x, results1.predict(x), color='red', label='liniowa, $S_{med}$ = '+"{:.2f}".format(S_lin_med))
    

    popt, pcov = curve_fit(power_func, x, y)
    ssr3 = np.sum(pow(y-power_func(x, *popt), 2))
    S_pow = (ssr3/(x.size-2))**(1/2)
    print("ssr_pow = "+str(ssr3))
    print("df_resid_pow = "+str(x.size-2))
    S_pow_div = S_pow/y.mean()
    S_pow_med = S_pow/y.median()    
    plt.plot(x, power_func(x, *popt), color='green', label='pow, $S_{med} = $'+"{:.2f}".format(S_pow_med)+', $\lambda$ = '+ "{:.2f}".format(popt[1]))
    
    xlog = np.log10(x)
    ylog = np.log10(y)
    x = sm.add_constant(x)
    xlog = sm.add_constant(xlog)
    
    model2 = sm.OLS(ylog,xlog)
    results2 = model2.fit()
    y_pred2 = pow(10, results2.predict(xlog))
    ssr2 = np.sum(pow(y-y_pred2, 2))
    S_log = (ssr2/results2.df_resid)**(1/2)
    print("ssr_log = "+str(ssr2))
    print("df_resid_log = "+str(results2.df_resid))
    S_log_div = S_log/y.mean()
    S_log_med = S_log/y.median()
    #print("Mediana_log = "+str(y.median()))
    
    plt.plot(x.loc[:, x.columns != 'const'], y_pred2, color='black', label='log, $S_{med}$ = '+"{:.2f}".format(S_log_med)+', $\lambda$ = '+ "{:.2f}".format(results2.params[1]))
    plt.legend()
    
    return results2.params[1], results2.params[0], results2.bse[1], results1.rsquared, results2.rsquared, S_lin, S_log, S_lin_div, S_log_div, S_lin_med, S_log_med

In [18]:
#dopasowanie liniowe do wyników w skali logarytmicznej log(y)=a*log(x)+b i liniowe y=ax i liniowe y=B*x^lambda na jednym wykresie, statsmodels
def fit_n_plot_lin_n_log_n_pow_plotlin(x, y, title, xscale_label, yscale_label):
    
    idx = np.isnan(y)
    y = y[~idx]
    x = x[~idx]
    idx = np.argsort(x)
    x_sorted = pd.Series(np.array(x)[idx])

    plt.plot(x, y, 'bo', markersize=3, label='dane')
    plt.title(title)
    #plt.yscale("log")
    #plt.xscale("log")
    plt.xlabel(xscale_label)
    plt.ylabel(yscale_label)
    
    #x = sm.add_constant(x[~idx]) #comment to not include intercept
    model1 = sm.OLS(y,x)
    results1 = model1.fit()
    S_lin = (results1.ssr/results1.df_resid)**(1/2)
    #print("ssr_lin = "+str(results1.ssr))
    #print("df_resid_lin = "+str(results1.df_resid))
    S_lin_div = S_lin/y.mean()
    S_lin_med = S_lin/y.median()
    #print("S_lin_med = "+str(S_lin_med))
    #print("Mediana_lin = "+str(y.median()))
#     plt.plot(x, results1.predict(x), color='red', label='liniowa, $R^2 = $'+"{:.2f}".format(results1.rsquared))
    #plt.plot(x, results1.predict(x), color='red', label='liniowa, $S_{med}$ = '+"{:.2f}".format(S_lin_med))
    

    popt, pcov = curve_fit(power_func, x, y)
    ssr3 = np.sum(pow(y-power_func(x, *popt), 2))
    S_pow = (ssr3/(x.size-2))**(1/2)
    #print("ssr_pow = "+str(ssr3))
    #print("df_resid_pow = "+str(x.size-2))
    S_pow_div = S_pow/y.mean()
    S_pow_med = S_pow/y.median()
    plt.plot(x_sorted, pd.Series(np.array(power_func(x, *popt))[idx]), '-', color='green', label='pow, $S_{med} = $'+"{:.2f}".format(S_pow_med)+', $\lambda$ = '+ "{:.2f}".format(popt[1]))
    
    
    xlog = np.log10(x)
    ylog = np.log10(y)
    x = sm.add_constant(x)
    x_sorted = sm.add_constant(x_sorted)
    xlog = sm.add_constant(xlog)
    
    model2 = sm.OLS(ylog,xlog)
    results2 = model2.fit()
    y_pred2 = pow(10, results2.predict(xlog))
    ssr2 = np.sum(pow(y-y_pred2, 2))
    S_log = (ssr2/results2.df_resid)**(1/2)
    #print("ssr_log = "+str(ssr2))
    #print("df_resid_log = "+str(results2.df_resid))
    S_log_div = S_log/y.mean()
    S_log_med = S_log/y.median()
    #print("Mediana_log = "+str(y.median()))
    #print(x.loc[:, x.columns != 'const'])
    #print(y_pred2)
    pd.Series(np.array(y_pred2)[idx])
    plt.plot(x_sorted.loc[:, x_sorted.columns != 'const'], pd.Series(np.array(y_pred2)[idx]), '-', color='black', label='log, $S_{med}$ = '+"{:.2f}".format(S_log_med)+', $\lambda$ = '+ "{:.2f}".format(results2.params[1]))
    plt.legend()
    
    return results2.params[1], results2.params[0], results2.bse[1], results1.rsquared, results2.rsquared, S_lin, S_log, S_lin_div, S_log_div, S_lin_med, S_log_med

In [19]:
def mean_date_from_pandemic_day(mean, n_days):
    return (mean + np.timedelta64(n_days, 'D'))

In [20]:
def mean_pandemic_day_from_date(mean, date):
    n_days = date - mean
    return (n_days)

In [21]:
def datetime_to_float(d):
    epoch = datetime.datetime.utcfromtimestamp(0)
    total_seconds =  (d - epoch).total_seconds()
    # total_seconds will be in decimals (millisecond precision)
    return total_seconds

In [22]:
# funkcja porównująca listy, była kiedyś potrzebna
def compare(s, t):
    t = list(t)   # make a mutable copy
    try:
        for elem in s:
            t.remove(elem)
    except ValueError:
        return False
    return t